In [ ]:
import sys
sys.path.insert(0,'..')

from src.models import *
from src.loss_functions import *
from src.noise import *
from src.metrics import *
from src.plotting import *
from src.generate_data import *
from src.real_data import *

import sklearn
import pandas as pd

from scipy.stats import bernoulli

from operator import xor
from sklearn.preprocessing import StandardScaler

import pickle as pkl

In [ ]:
dataset = "cshock_mimic"

X_train, X_test, y_train, y_test, group_train, group_test = load_dataset_splits(dataset, group = "age")

    
print(X_train.shape)
print(y_test.shape)

In [ ]:
noise_levels = [0]
noise_type = "class_independent"
# noise_transition_dict = {0: np.array([[0.75,0.25],
#                                       [0.25,0.75]]),
#                          1: np.array([[0.75,0.25],
#                                       [0.25,0.75]])}
fixed_class = 1
fixed_noise = 0.0

model_type = "NN"
uncertainty_type = "forward"
batch_size = 512

max_iter = 10000
m = 1000
d= X_train.shape[1]
noise_level = 0.4


misspecify = False

y_train = y_train.astype(int)
y_test = y_test.astype(int)

p_y_x_dict =  calculate_prior(y_train, noise_type = noise_type, group=group_train) 


In [ ]:
_, T_true = generate_class_independent_noise(y_train, noise_level) #Fixed noise draw


if misspecify == True: #Misspecified T
    pass #TODO
    T_est = None
else: #Correct T
    T_est = T_true



In [ ]:
metrics = simulate_noise_and_train_model(m, max_iter,  X_train, y_train, X_test, y_test, p_y_x_dict, T_true = T_true, T_est = T_est, noise_type = noise_type, model_type = model_type, uncertainty_type=uncertainty_type,  fixed_class=fixed_class, fixed_noise=fixed_noise, batch_size = batch_size)


In [ ]:
T_dicts = {"case1": {0: np.array([[0.8,0.2],
                            [0.2,0.8]]),
                1: np.array([[0.6,0.4],
                            [0.4,0.6]])}, 
                "case2": { 0: np.array([[0.75,0.25],
                                [0.25,0.75]]),
                    1: np.array([[0.75,0.25],
                                [0.25,0.75]])},
                "case3": { 0: np.array([[0.75,0.25],
                                [0.25,0.75]]),
                    1: np.array([[0.75,0.25],
                                [0.25,0.75]])},              
                "case4": { 0: np.array([[0.75,0.25],
                                [0.25,0.75]]),
                    1: np.array([[0.75,0.25],
                                [0.25,0.75]])}               
    }

In [ ]:
for case, T_dict in T_dicts.items():
    print(case)
    

In [8]:
yn_train, T_true = generate_group_noise(y_train, group_train, T_dict)

noise_type = "group"

if uncertainty_type == "forward":
    y_vec = y_train
else: # backward
    y_vec = yn_train

if misspecify == True: #Misspecified T
    pass #TODO
else: #Correct T
    T_est = T_true

# Generate error rates
metrics = simulate_noise_and_train_model(   m, 
                                            max_iter, 
                                            X_train,
                                            y_train,
                                            y_vec, 
                                            X_test, 
                                            y_test,
                                            p_y_x_dict,
                                            T_true = T_true,
                                            T_est = T_est,
                                            group_train = group_train,
                                            group_test = group_test,
                                            noise_type = noise_type, 
                                            model_type = model_type, 
                                            uncertainty_type=uncertainty_type, 
                                            T_dict = T_dict,
                                            batch_size = batch_size)


path = os.path.join(files_path, f"{case}_metrics.pkl")

        # Open a file for writing in binary mode
with open(path, 'wb') as file:
    # Use pickle to write the dictionary to the file
    pkl.dump(metrics, file)

TypeError: simulate_noise_and_train_model() got an unexpected keyword argument 'group_train'

In [ ]:
T_true